## Imports

In [4]:
# Data analysis packages:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.3f'%x)
#from datetime import datetime as dt

# Visualization packages:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

# import warnings
import itertools
import datetime as dt
from IPython.display import HTML # to see everything
plt.style.use('seaborn-darkgrid')
# warnings.filterwarnings("ignore")

Import sklearn and statsmodels

In [5]:
# import sklearn 
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error

Import from Keras

In [6]:
# !pip install tf-nightly-2.0-preview
import tensorflow as tf

# import Sequential
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers import Input, Dense, LSTM, Embedding, Dropout
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import (EarlyStopping, ModelCheckpoint,
                                               TensorBoard, ReduceLROnPlateau, CSVLogger)
# print 
print(tf.__version__)
print(tf.keras.__version__)

2.0.0-beta0
2.2.4-tf


The shape of the input for making a single prediction must be 1 sample, 3 time steps, and 3 features, or [1, 3, 3]

In [ ]:
# import plots as p
# import function as f
# import data_testing as dt
# import helper_functions as hf
%load_ext autoreload
%autoreload 2

In [ ]:
# define model
# n_features = 3
# model = Sequential()
# model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
# model.add(LSTM(100, activation='relu'))
# model.add(Dense(n_features))
# model.compile(optimizer='adam', loss='mse')

## Multiple Parallel Input and Multi-Step Output

In [ ]:
# split a multivariate sequence into samples
from numpy import array
def (sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# Load the dataset

In [ ]:
# # convert series to supervised learning
# def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
#     n_vars = 1 if type(data) is list else data.shape[1]
#     df = DataFrame(data)
#     cols, names = list(), list()
#     # input sequence (t-n, ... t-1)
#     for i in range(n_in, 0, -1):
#         cols.append(df.shift(i))
#         names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
#     # forecast sequence (t, t+1, ... t+n)
#     for i in range(0, n_out):
#         cols.append(df.shift(-i))
#         if i == 0:
#             names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
#         else:
#             names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
#     # put it all together
#     agg = concat(cols, axis=1)
#     agg.columns = names
#     # drop rows with NaN values
#     if dropnan:
#         agg.dropna(inplace=True)
#     return agg
 

In [ ]:
office = pd.read_csv('data_folder/office_1249.csv', index_col=['timestamp'], parse_dates=['timestamp'])
usecols =['meter', 'meter_reading', 'air_temperature', 'cloud_coverage', 'dew_temperature','precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
# office = office[office[usecols]]
office = office[usecols]
office_temp = office.ffill()
office_data = office_temp.bfill()
office_data.head(3)

In [ ]:
elec_consumption = office_data[office_data['meter']==0]
elec_consumption.drop(columns=['meter'], inplace=True)
print(elec_consumption.shape)
elec_consumption.head()

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# import numpy as np
dataset = elec_consumption['meter_reading'].values
scaler = MinMaxScaler(feature_range=(0, 1))
print('Min and Max X data')
print('    Min:', np.min(dataset))
print('    Max:', np.max(dataset))
dataset = dataset.reshape(-1,1)
print(dataset.shape)

In [ ]:
dataset

In [ ]:
# training set 
scaled_data = scaler.fit_transform(dataset)
print('Min and Max X_train_scaled data')
print('    Min:', np.min(scaled_data))
print('    Max:', np.max(scaled_data))
print(scaled_data.shape)
scaled_data

In [ ]:
# split to train and test 
train_size = int(len(dataset) * 0.80)
test_size = len(dataset) - train_size
train, test = scaled_data[0:train_size,:], scaled_data[train_size:len(dataset),:]
# x_train_scaled, x_test_scaled = dataset_scaled[0:train_size,:], dataset_scaled[train_size:len(dataset),:]
print(train_size)
print(train.shape, test.shape)
train

In [ ]:
# dataset = elec_consumption.values
# print(scaled_data.shape)
n_steps_in, n_steps_out = 24,1
x_scaled_train, y_scaled_train = split_sequences(train, n_steps_in, n_steps_out)
# x_train_scaled, y_train_scaled = split_sequences(x_train_scaled, n_steps_in, n_steps_out)
x_scaled_test, y_scaled_test = split_sequences(test, n_steps_in, n_steps_out)
# x_test_scaled, y_test_scaled = split_sequences(x_test_scaled, n_steps_in, n_steps_out)
print(x_scaled_train.shape, y_scaled_train.shape)
print(x_scaled_test.shape, y_scaled_test.shape)

# summarize the data 
# for i in range(len(X)):
#     print(X[i], y[i])

In [ ]:
y_scaled_train[0]

In [ ]:
x_scaled_train[0]

In [ ]:
# print(x_train.shape)
# print(x_train_scaled.shape)

### Model Architecture

In [ ]:
# Model Architecture
num_x_signals = x_scaled_train.shape[2] # 1
print(num_x_signals)
num_y_signals = y_scaled_train.shape[2] # 1
print(num_y_signals)

#### Initalize the RNN

In [ ]:
# clear a model
tf.keras.backend.clear_session()

In [ ]:
# Initializing RNN 
model = Sequential()

#### Adding the LSTM layers and some Dropout regularization

In [ ]:
x_scaled_train.shape[1],1

In [ ]:
# Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units=50, return_sequences = True,
                   input_shape=(n_steps_in,1))) # or 24
model.add(Dropout(0.2))
# model.summary()

In [ ]:
# adding the second LSTM and some Dropout regularization
model. add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
# model.summary()

In [ ]:
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
# model.summary()

In [ ]:
# adding the output layer
model.add(Dense(units=num_y_signals))
# model.summary()
  
ggplot(compare_cx, aes(x = rowname, y = value, color = type)) +
  geom_line() +
  xlab("epoch") +
  ylab("loss")

In [ ]:
compare_cx <- data.frame(
  baseline_train = baseline_history$metrics$loss,
  baseline_val = baseline_history$metrics$val_loss,
  smaller_train = smaller_history$metrics$loss,
  smaller_val = smaller_history$metrics$val_loss,
  bigger_train = bigger_history$metrics$loss,
  bigger_val = bigger_history$metrics$val_loss
) %>%

#### Callback functions

In [ ]:
# import os 
# for i in ['logs', 'weights']:
#     if not os.path.exists(i):
#         os.makedirs(i)

In [ ]:
path_checkpoint = 'weights/model_weights.best.{epoch:02d}-{val_loss:.2f}.hdf5'
# 23_checkpoint.keras'
# model check point
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                     monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)
# Early stopping 
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=5, verbose=1)
# tensor board 
callback_tensorboard = TensorBoard(log_dir='./23_logs/',
                                   histogram_freq=0,
                                   write_graph=False)
# reduce LR on Plateau
callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1,
                                       min_lr=1e-4,
                                       patience=0,
                                       verbose=1)

csv_logger = CSVLogger('logs/training_model.csv')
# callback list 
callback = [callback_early_stopping,
           callback_checkpoint,
           callback_tensorboard, 
           callback_reduce_lr, 
           csv_logger]

#### Compile the RNN

In [ ]:
# compiling the RNN
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['acc'])
model.summary()

#### Fit the RNN to the training set

In [ ]:
print(x_scaled_train.shape)
print(y_scaled_train.shape)
print('-----')
print(x_scaled_test.shape)
print(y_scaled_test.shape)
type(x_scaled_train)

In [ ]:
# y_train = []
# for i in range(24, 2035):
#     X_train.append(training_set_scaled[i-60:i, 0])
#     y_train.append(training_set_scaled[i, 0])
# X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
# model.fit(x_scaled_train, y_scaled_train.ravel(), epochs=3, batch_size=64, callbacks=callback,
#           validation_data=(x_scaled_test, y_scaled_test.ravel()))

In [ ]:
y_test = y_scaled_test.ravel()
y_train = y_scaled_train.ravel()
print(y_test.shape)
print(y_train.shape)

In [ ]:
[y_train][0].shape

In [ ]:
history = model.fit(x_scaled_train, y_train, epochs=3, batch_size=32, callbacks=callback,
          validation_data=(x_scaled_test, y_test), verbose=1, shuffle=False)

In [ ]:
# training phase
# model.summary()

In [ ]:
# make predictions
train_predict = model.predict(x_scaled_train)
test_predict = model.predict(x_scaled_test)
print(train_predict.shape)
print(test_predict.shape)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
# invert predictions
train_predict = scaler.inverse_transform(train_predict)
Y_train = scaler.inverse_transform([y_train])
test_predict = scaler.inverse_transform(test_predict)
Y_test = scaler.inverse_transform([y_test])

print('Train Mean Absolute Error:', mean_absolute_error(Y_train[0], train_predict[:,0]))
print('Train Root Mean Squared Error:',np.sqrt(mean_squared_error(Y_train[0], train_predict[:,0])))
print('Test Mean Absolute Error:', mean_absolute_error(Y_test[0], test_predict[:,0]))
print('Test Root Mean Squared Error:',np.sqrt(mean_squared_error(Y_test[0], test_predict[:,0])))
# print(train_predict.shape)
# print(test_predict.shape)

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(loc='upper right')
plt.show();

In [ ]:
aa=[x for x in range(200)]
plt.figure(figsize=(8,4))
plt.plot(aa, Y_test[0][:200], marker='.', label="actual")
plt.plot(aa, test_predict[:,0][:200], 'r', label="prediction")
# plt.tick_params(left=False, labelleft=True) #remove ticks
plt.tight_layout()
sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();

In [ ]:
test_predict

In [ ]:
# save the model in h5 format
model.save('weights/first_model.h5')

In [ ]:
# load the model
sisay = load_model('weights/first_model.h5')

In [ ]:
# evaluate the model
sisay.evaluate(x_train_scaled, y_train_scaled.ravel())

In [ ]:
# sisay.evaluate(x_test_scaled, y_test_scaled)

In [ ]:
print(x_train_scaled.shape)
print(type(x_train_scaled))
x_test_scaled.shape[0]

In [ ]:
x_test_scaled.shape

In [ ]:
x_test_ex = np.reshape(x_test_scaled, (x_test_scaled.shape[0], x_test_scaled.shape[1], 1))
print(x_test_ex.shape)

In [ ]:
x_test_ex[:,0,:]

In [ ]:
predicted_energy = model.predict(x_test_scaled)
print(predicted_energy.shape)

In [ ]:
# predicted_energy = np.reshape(x_test_scaled, (x_test_scaled.shape[0], :,x_test_scaled.shape[2]))
predicted_energy = np.reshape(x_train_scaled, (x_test_scaled.shape[0], x_test_scaled[::-1]))
# np.reshape((-1,2))
print(predicted_energy.shape)

In [ ]:
predicted_energy = scaler.inverse_transform(predicted_energy)
print(predicted_energy.shape)

In [ ]:
predicted_energy

In [ ]:
# print(x_test_scaled.shape)
# x_test = scaler.inverse_transform(x_test_scaled)
print(x_test.shape)

In [ ]:
# Visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Real TATA Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted TAT Stock Price')
plt.title('TATA Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('TATA Stock Price')
plt.legend()
plt.show()

In [ ]:
result = model.evaluate(x=np.expand_dims(x_test_scaled, axis=0),
                       y=np.expand_dims(y_test_scaled, axis=0))

# From an awesome code

In [ ]:
elec_consumption.head()

In [ ]:
df_10

In [ ]:
target_names = ['meter_reading', 'air_temperature']#, 'dew_temperature'] # 3 columns
df_target = elec_consumption[target_names]
# print(df_target.shape)
df_10 = df_target[:10]

In [ ]:
target_data = df_10.values
print(target_data.shape)

In [ ]:
x_data, y_data = split_sequences(target_data, 3, 1)

In [ ]:
x_data.shape # 3 is the shift and 2 is a sequance

In [ ]:
y_data.shape

In [ ]:
train_split = 0.9
num_data = len(x_data)
# print(num_data)
num_train = int(train_split * num_data)
# print(num_train)
num_test = int(num_data - num_train)
# print(num_test)

# X_tain and X_test
x_train = x_data[0:num_train]
x_test = x_data[num_train:]
print(x_train.shape)
print(x_test.shape)

# y_train and y_test 
y_train = y_data[0:num_train]
y_test = y_data[num_train:]
print(y_train.shape)
print(y_test.shape)

In [ ]:
num_x_signals = x_data.shape[1]
print(num_x_signals)

num_y_signals = y_data.shape[1]
print(num_y_signals)

In [ ]:
print("Min:", np.min(x_train))
print("Max:", np.max(x_train))

In [ ]:
x_scaler = MinMaxScaler()

In [ ]:
# x_train_scaled = x_scaler.fit_transform(x_train)

In [ ]:
x_train

In [ ]:
x_train.shape

In [ ]:
.shape
# a.reshape(-1,3)

In [ ]:
model.add(LSTM(units=100,
              return_sequences=True,
              input_shape=(None, num_x_signals,)))

model.add(Dense(num_y_signals, activation='sigmoid'))

# summary 
model.summary()

In [ ]:
# %load -r 135-194 helper_functions.py

### Train the RNN

In [ ]:
%%time 
# generator = batch_generator(batch_size, sequence_length)
# x_batch, y_batch = next(generator)
model.fit(X_train, y_train, batch_size=72, epochs=20) #, verbose=1,
#          callbacks=, validation_data)

In [ ]:
X_train.shape
y_train.shape[0]

In [ ]:
# history = model.fit(X_train, new, epochs=20, batch_size=70,
#                     callbacks=[EarlyStopping(monitor='val_loss', patience=10)],
#                     verbose=1, shuffle=False, 
#                     validation_data=(X_test[1], y_test[2]),
# # Training Phase
# model.summary()

In [ ]:
# history['loss']

In [ ]:
import matplotlib.pyplot as plr
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(loc='upper right')
plt.show();

In [ ]:
# test set 
x_test_scaled = x_scaler.transform(x_test)
print('Min and Max X_test data')
print('    Min:', np.min(x_test))
print('    Max:', np.max(x_test))

In [ ]:
model = Sequential()
model.add(LSTM(units=100,
              return_sequences=True,
              input_shape=(None, num_x_signals,)))

In [ ]:
model.add(Dense(num_y_signals, activation='sigmoid'))

In [ ]:
if False:
    from tensorflow.python.keras.initializers import RandomUniform

    # Maybe use lower init-ranges.
    init = RandomUniform(minval=-0.05, maxval=0.05) #change the values to a smaller

    model.add(Dense(num_y_signals,
                    activation='linear',
                    kernel_initializer=init))
model.summary()

In [ ]:
# optimization  
optimizer = RMSprop(lr=1e-3)
# optimizer = tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9)
# model.compile(loss=loss_mse_warmup, optimizer=optimizer)
model.compile(loss='mse'
              , optimizer=optimizer)

In [ ]:
%%time
model.fit(x=X_train, y=y_train,
                    epochs=20,
                    steps_per_epoch=100)#,
#                     validation_data=validation_data,
#                     callbacks=callbacks)

## LSTMs